In [ ]:
%load_ext autoreload
%autoreload 2
    
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from glob import glob
from waffles.data_classes.EasyWaveformCreator import EasyWaveformCreator
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.np02_utils.AutoMap import dict_endpoints_channels_list, dict_module_to_uniqch, dict_uniqch_to_module, strUch
from waffles.np02_utils.AutoMap import ordered_channels_cathode, ordered_channels_membrane, ordered_modules_cathode, ordered_modules_membrane
from waffles.np02_utils.PlotUtils import matplotlib_plot_WaveformSetGrid
from waffles.np02_utils.load_utils import ch_read_template
from DeconvFitterVDWrapper import DeconvFitterVDWrapper

# from utils_conv import process_convfit, plot_convfit
from larfitter import retrieve_responses
from utils_plot import plot_fit

import mplhep
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 20,
                     'grid.linestyle': '--',
                     'axes.grid': True,
                     'figure.autolayout': True,
                     'figure.figsize': [14,6]
                     })


In [ ]:
run = 39510 

response_root_path = "/eos/experiment/neutplatform/protodune/experiments/ProtoDUNE-VD/xenon_averages/average_waveforms-hvieirad/"


response_folder_cathode = Path(response_root_path + f'run{run:06d}_cathode_response')
response_folder_membrane = Path(response_root_path + f'run{run:06d}_membrane_response')

In [ ]:
templates = ch_read_template('templates_large_pulses')

In [ ]:
responses = {}
counts = {}
retrieve_responses(response_folder_cathode, responses, counts)
retrieve_responses(response_folder_membrane, responses, counts)

In [ ]:
dfitparams = dict(
    scinttype = 'lar',
    error = 0.05,
    filter_type = 'Gauss',
    cutoff_MHz = 2.5,
    dtime = 16,
)
cfit = {}
dict_ep_ch_with_both_signals = {}
for ep, ch_wvf in responses.items():
    for ch, _ in ch_wvf.items():
        if ep in templates.keys():
            cfit[ep] = cfit.get(ep, {})
            if ch in templates[ep].keys():
                dict_ep_ch_with_both_signals[ep] = dict_ep_ch_with_both_signals.get(ep, []) + [ch]
                cfit[ep][ch] = DeconvFitterVDWrapper(**dfitparams)

In [ ]:
dict_ok_membrane = { 107: dict_ep_ch_with_both_signals[107] if 107 in dict_ep_ch_with_both_signals else [] }
dict_ok_cathode = { 106: dict_ep_ch_with_both_signals[106] if 106 in dict_ep_ch_with_both_signals else []}

In [ ]:
wfset = EasyWaveformCreator.create_WaveformSet_dictEndpointCh(dict_endpoint_ch=dict_endpoints_channels_list, run_number=run)

In [ ]:
from waffles.utils.time_align_utils import find_threshold_crossing
module_test = "C3(1)" # ex: C2(2)

if module_test:
    ep = dict_module_to_uniqch[module_test].endpoint
    ch = dict_module_to_uniqch[module_test].channel
else:
    ep = 106
    ch = 30

tmpcfit = cfit[ep][ch]
tmpcfit.set_template_waveform(templates[ep][ch])
tmpcfit.set_response_waveform(responses[ep][ch])

tmpcfit.response = tmpcfit.response - np.mean(tmpcfit.response[:50])
tmpcfit.template = tmpcfit.template - np.mean(tmpcfit.template[200:240])
plt.plot(tmpcfit.template, label='template')
plt.plot(tmpcfit.response*np.max(tmpcfit.template)/np.max(tmpcfit.response), label='normalized-response')
plt.title(f"{dict_uniqch_to_module[strUch(ep,ch)]}: {ep}-{ch}")
plt.legend()


cross_template = find_threshold_crossing(tmpcfit.template, 0.5)
shift = cross_template
tmpcfit.getFFTs()
plt.figure()
deconvolved = tmpcfit.backFFT(tmpcfit.deconvolve(tmpcfit.response, tmpcfit.template))
deconvolved = np.roll(deconvolved,int(shift))
plt.plot(deconvolved, label='deconvolued')
plt.yscale('log')
plt.legend()

In [ ]:
paramsplot = {
    'cfit':cfit,
    'dofit':True,
    'responses': responses,
    'templates': templates,
    'verbose': False,
}

fig, axs = matplotlib_plot_WaveformSetGrid(wfset, detector=ordered_modules_cathode, plot_function=plot_fit, func_params=paramsplot, cols=4, figsize=(32,32))

Path('./graphs').mkdir(exist_ok=True)
plt.savefig(f"./graphs/deconvfit_results_cathode_run{run:06d}")
plt.show()


In [ ]:
fig, axs = matplotlib_plot_WaveformSetGrid(wfset, detector=ordered_modules_membrane, plot_function=plot_fit, func_params=paramsplot, cols=4, figsize=(32,32))
plt.savefig(f"./graphs/deconvfit_results_membrane_run{run:06d}")
plt.show()